<a href="https://colab.research.google.com/github/Danny2173/RAGproject/blob/main/2_Question_Answer_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Imports and Installs

In [ ]:
# Install dependencies
%pip install -q transformers openai tqdm

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
import json, time
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from tqdm import tqdm
import openai


Mounted at /content/drive


##Loading Corpus

In [ ]:
load_path = '/content/drive/MyDrive/corpus.json'

# Load the corpus
with open(load_path, "r") as f:
    corpus = json.load(f)

print("Corpus loaded")


Corpus loaded


In [ ]:
# Preview formatted text
for i, doc in enumerate(corpus[:7]):
    print(f"Document {i+1}")
    print(doc["text"])
    print("-" * 100)


##Generating questions and answers based on the context passages

In [ ]:
# Load FLAN-T5 model
model = "google/flan-t5-large"
qa_tokenizer = AutoTokenizer.from_pretrained(model)
qa_model = AutoModelForSeq2SeqLM.from_pretrained(model).to("cuda")

# Initialise
synthetic_dataset = []

# Create one pair per chunk, with a progress bar
for chunk in tqdm(corpus, desc="Generating qa pairs"):
    context = chunk["text"]

    # Generate question
    prompt_q = f"You are a medical question writer. Based on the passage below, generate a clear, specific, factual question that could be answered from the passage. Passage: {context}"
    inputs_q = qa_tokenizer(prompt_q, return_tensors="pt", truncation=True, max_length=512).to("cuda")
    outputs_q = qa_model.generate(**inputs_q, max_length=64)
    question = qa_tokenizer.decode(outputs_q[0], skip_special_tokens=True)

    # Generate answer
    prompt_a = f"Based on the text:\n{context}\nAnswer this question: {question}"
    inputs_a = qa_tokenizer(prompt_a, return_tensors="pt", truncation=True, max_length=512).to("cuda")
    outputs_a = qa_model.generate(**inputs_a, max_length=256)
    answer = qa_tokenizer.decode(outputs_a[0], skip_special_tokens=True)

    # Save the qa pair
    synthetic_dataset.append({
        "context": context,
        "question": question,
        "answer": answer
    })

    # time.sleep(0.3)  # prevent GPU overload

with open("synthetic_dataset.json", "w") as f:
    json.dump(synthetic_dataset, f, indent=2)

In [ ]:
for i, item in enumerate(synthetic_dataset[:10], 1):
    print(f"Example {i}")
    print("Context:", item["context"][:200].strip().replace("\n", " "))
    print("Question:", item["question"])
    print("Answer:", item["answer"])


##Saving synthetic dataset

In [ ]:
save_path = '/content/drive/MyDrive/synthetic_dataset.json'

# Save as JSON
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(synthetic_dataset, f, ensure_ascii=False, indent=2)

print(f"Saved to {save_path}")

Saved to /content/drive/MyDrive/synthetic_dataset.json


In [ ]:
load_path = '/content/drive/MyDrive/synthetic_dataset.json'

# Load JSON file
with open(load_path, "r", encoding="utf-8") as f:
    synthetic_dataset = json.load(f)

print("Loaded qa pairs")

Loaded qa pairs


##Expanding the answers to form longer, grammatically correct sentences.

In [ ]:
# Set your OpenAI API key
openai.api_key = "REPLACE_WITH_YOUR_OPENAI_API_KEY"

In [ ]:
# OpenAI client
client = openai.OpenAI(
    api_key=openai.api_key
)

# Expnading each answer
expanded_data = []
for i, item in enumerate(tqdm(synthetic_dataset, desc="Expanding answers")):
    question = item["question"]
    short_answer = item["answer"]
    context = item["context"]

    prompt = (
        f"Context:\n{context}\n\n"
        f"Question: {question}\n"
        f"Short Answer: {short_answer}\n\n"
        f"Rewrite the short answer into a full-sentence answer that directly responds to the question."
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.4,
    )

    full_answer = response.choices[0].message.content.strip()

    if (i + 1) % 100 == 0 or i < 5:
        print(f"Example {i+1}")
        print("Question:", question)
        print("Short Answer:", short_answer)
        print("Expanded Answer:", full_answer)

    item["answer"] = full_answer
    expanded_data.append(item)

with open("expanded_dataset.json", "w", encoding="utf-8") as f:
    json.dump(expanded_data, f, indent=2, ensure_ascii=False)

Expanding answers:   0%|          | 1/5121 [00:02<3:35:54,  2.53s/it]

Example 1
Question: What is the name for dry, dark patches of skin that usually appear in the armpits, neck or groin?
Short Answer: Acanthosis nigricans
Expanded Answer: Acanthosis nigricans is the name for dry, dark patches of skin that usually appear in the armpits, neck, or groin.


Expanding answers:   0%|          | 2/5121 [00:03<2:36:14,  1.83s/it]

Example 2
Question: What is the most common cause of acanthosis nigricans?
Short Answer: obesity
Expanded Answer: Obesity is the most common cause of acanthosis nigricans.


Expanding answers:   0%|          | 3/5121 [00:05<2:44:48,  1.93s/it]

...

Expanding answers:  98%|█████████▊| 5000/5121 [1:13:49<01:34,  1.29it/s]

Example 5000
Question: What are the main symptoms of vaginal cancer?
Short Answer: a lump in the vagina ulcers and other skin changes in or around the vagina
Expanded Answer: The main symptoms of vaginal cancer include the presence of a lump in the vagina and ulcers and other skin changes in or around the vagina.


Expanding answers: 100%|█████████▉| 5100/5121 [1:15:02<00:13,  1.51it/s]

Example 5100
Question: What can be done to help prevent infecting others?
Short Answer: wash your hands with soap and water after going to the toilet or changing nappies
Expanded Answer: To help prevent infecting others, you should wash your hands with soap and water after going to the toilet or changing nappies.


Expanding answers: 100%|██████████| 5121/5121 [1:15:16<00:00,  1.13it/s]


##Saving Expanded dataset

In [ ]:
save_path = "/content/drive/MyDrive/expanded_dataset.json"

# Save as JSON
with open(save_path, "w", encoding="utf-8") as f:
    json.dump(expanded_data, f, indent=2, ensure_ascii=False)

print(f"Saved to {save_path}")

Saved to /content/drive/MyDrive/expanded_dataset.json
